In [1]:
import pandas as pd
import geopandas
from folium import Circle, Marker, Icon, Map
from pymongo import MongoClient, GEOSPHERE
conn = MongoClient("localhost:27017")
db = conn.get_database("madrid")
import requests
import unicodedata 
import json
#from dotenv import load_dotenv
#import os

## Distrito -> coordenadas

### Pueblo

In [2]:
df_madrid = pd.read_csv("data/df_madrid.csv",index_col=0)

In [3]:
df_madrid.head()

,provincia,poblacion,CP,lat,lon
0,Madrid,Ajalvir,28864,-3.480.710.160,40.535.068.130
1,Madrid,Alameda del Valle,28742,-3.843.661.950,40.916.583.150
2,Madrid,El Alamo,28607,-3.990.797.560,40.228.256.070
3,Madrid,El Alamo,28807,-3.990.797.560,40.228.256.070
4,Madrid,Alcala de Henares,28801,-3.369.251.020,40.480.332.320


In [4]:
municipios = ['Móstoles', 'Alcalá de Henares','Fuenlabrada',
'Leganés','Getafe','Alcorcón','Torrejón de Ardoz',
'Parla','Alcobendas', 'Las Rozas de Madrid', 'San Sebastián de los Reyes',
'Rivas-Vaciamadrid','Pozuelo de Alarcón', 'Coslada', 'Valdemoro',
'Majadahonda' , 'Collado Villalba', 'Aranjuez', 'Arganda del Rey',
'Boadilla del Monte', 'Pinto','Colmenar Viejo','Tres Cantos',
'San Fernando de Henares','Galapagar','Arroyomolinos','Navalcarnero',
'Villaviciosa de Odón','Paracuellos de Jarama','Ciempozuelos','Torrelodones',
'Mejorada del Campo','Villanueva de la Cañada','Algete','Humanes de Madrid',
'San Martín de la Vega','San Lorenzo de El Escorial','Villanueva del Pardillo', ' El Escorial']

### Distritos

In [5]:
df_distritos = pd.read_csv("data/df_districts.csv",index_col=0)

In [6]:
df_distritos.head()

,Distrito,CP
0,FUENCARRAL-EL PARDO,28050
1,HORTALEZA,28050
2,FUENCARRAL-EL PARDO,28108
3,CIUDAD LINEAL,28027
4,HORTALEZA,28027


In [7]:
lista_distritos = ['fuencarral','el pardo', 'hortaleza', 'ciudad lineal', 'san blas','canillejas', 'barajas', 'moratalaz', 'puente de vallecas', 'vicalvaro', 'villa de vallecas', 'villaverde', 'usera', 'carabanchel', 'latina', 'moncloa','aravaca', 'centro', 'tetuan', 'chamartin', 'chamberi', 'retiro', 'arganzuela', 'salamanca']

### Distrito -> coordenadas

In [8]:
district = input("Selecciona un barrio/municipio de Madrid: ")

Selecciona un barrio/municipio de Madrid: Chamartín


In [9]:
def barrio_a_coordenadas(df_madrid,df_distritos, district):
    coordinates = {}
    district = unicodedata.normalize('NFD', district)\
    .encode('ascii', 'ignore')\
    .decode("utf-8")
    print(district)
    lista_distritos = ['fuencarral','el pardo', 'hortaleza', 'ciudad lineal', 'san blas',
                       'canillejas', 'barajas', 'moratalaz', 'puente de vallecas', 'vicalvaro',
                       'villa de vallecas', 'villaverde', 'usera', 'carabanchel', 'latina', 
                       'moncloa','aravaca', 'centro', 'tetuan', 'chamartin', 'chamberi', 'retiro',
                       'arganzuela', 'salamanca']
    if district.lower() in lista_distritos:
        district = district.upper()
        cp = 28037
        for index, row in df_distritos.iterrows():
            if district in row.Distrito:
                cp = row.CP
        datos = requests.get(f"https://geocode.xyz/{cp}?json=1").json()
        datos = datos["alt"]["loc"]
        for country in datos:
            if country['countryname'] == "Spain":
                coordinates["latt"] = country["latt"]
                coordinates["long"] = country["longt"]
    else:
        for index, row in df_madrid.iterrows():
            if district.upper() in row.poblacion.upper():
                coordinates["latt"] = row["lon"][:6]
                coordinates["long"] = row["lat"][:6]            
                break
    return coordinates

In [10]:
barrio_a_coordenadas(df_madrid,df_distritos,"Chamartín")

Chamartin
CHAMARTIN


{'latt': '40.49341', 'long': '-3.69538'}

## Geoquery

In [11]:
def geoquery(coordinates):
    collection = db.get_collection("todo")
    coord_point = {"type":"Point", "coordinates": [float(coordinates['latt']), float(coordinates['long'])]}
    query = {"geometry": {"$near": {"$geometry": coord_point,"$minDistance": 0, "$maxDistance": 1500}}}
    query_final = collection.find(query)
    df = pd.DataFrame((query_final))
    return df

In [12]:
df = geoquery(barrio_a_coordenadas(df_madrid,df_distritos,"las rozas de madrid"))

las rozas de madrid


In [13]:
df.head()

,_id,name,latitude,longitude,geometry,place,type
0,60a24ec25065d2cc63d9ab52,AV.IGLESIA-REAL,40.491814,-3.873164,"{'type': 'Point', 'coordinates': [40.491813659...",bus,transport
1,60a24ec20ffc96451a054a87,AV.IGLESIA-REAL,40.491814,-3.873164,"{'type': 'Point', 'coordinates': [40.491813659...",bus,transport
2,60a24ec25065d2cc63d9ab81,AV.CONSTITUCIÓN-AYUNTAMIENTO,40.492527,-3.874967,"{'type': 'Point', 'coordinates': [40.492527008...",bus,transport
3,60a24ec20ffc96451a054ac6,AV.CONSTITUCIÓN-AYUNTAMIENTO,40.492527,-3.874967,"{'type': 'Point', 'coordinates': [40.492527008...",bus,transport
4,60a24ec20ffc96451a054de7,CTRA.M505-BURGOCENTRO,40.490986,-3.876587,"{'type': 'Point', 'coordinates': [40.490985870...",bus,transport


## DF

In [54]:
def df_planes_bonito(df):
    df_plan = df[df["type"]=="plan"]
    df_plan = df_plan.set_index("name")
    try:
        df_plan = df_plan.drop(["_id","CP","price_level", "type","subtype", "latitude", "longitude", "geometry"],axis=1)
        df_plan = df_plan.drop_duplicates()
        return df_plan
    except: 
        pass
    

In [55]:
df_plan = df_planes_bonito(geoquery(barrio_a_coordenadas(df_madrid,df_distritos,"alcobendas")))

alcobendas


In [56]:
df_plan

,rating,address,place,price
name,,,,
Sala Liverpool Karaoke,3.4,"Plaza de Felipe Álvarez Gadea, 28100 Alcobenda...",karaoke,1-15€
Bonsai Museum,4.4,"Av. Olímpica, S/N, 28100 Alcobendas, Madrid, S...",museum,15-25€
Yellow Cab,4.0,"Av. Olímpica, 5, 28108 Alcobendas, Madrid, Spain",brewery,15-25€
Active Tourism Club Get Going,4.8,"Calle del Dos de Mayo, 6, 28703 San Sebastián ...",senderismo,Free
RANKING PADEL,0.0,"Calle Radio Utopía, 16, 28703 San Sebastián de...",pista padel,1-15€
Planet Reball,4.3,"Calle de Pedro Colmenar, 9, 28703 San Sebastiá...",paintball,15-25€
Community of Madrid Park,3.8,"Calle de Miraflores, 16, 28100 Alcobendas, Mad...",park,Free
Manila Look and Feel Good,4.1,"Av. Olímpica, 32, Local 9, 28108 Alcobendas, M...",spa,25-50€
Planet Bowling,1.0,"Calle de Anabel Segura, 7, 28108 Alcobendas, M...",bowling,1-15€


In [17]:
df.columns

Index(['_id', 'name', 'latitude', 'longitude', 'geometry', 'place', 'type'], dtype='object')

In [51]:
def df_tpte_bonito(df):
    df_tpte = df[df["type"]=="transport"]
    df_tpte = df_tpte.set_index("name")
    try:
        df_tpte = df_tpte.drop(["subtype","rating","_id","CP","price_level","address", "type", "latitude", "longitude", "geometry","price"],axis=1)
        df_tpte = df_tpte.drop_duplicates()
        df_tpte = df_tpte.rename(columns={"place":"mean of transport near"})
        df_tpte = df_tpte.set_index("mean of transport near")
        return df_tpte
    except:
        df_tpte = df_tpte.drop(['_id', 'latitude', 'longitude', 'geometry', 'type'],axis=1)
        df_tpte = df_tpte.drop_duplicates()
        df_tpte = df_tpte.rename(columns={"place":"mean of transport near"})
        df_tpte = df_tpte.set_index("mean of transport near")
        return df_tpte
    

In [52]:
df_tpte = df_tpte_bonito(df)

In [53]:
df_tpte

""
mean of transport near
bus
train


## MAP

In [ ]:
def mapita(df,coordinates):
    map_1 = Map(location=[float(coordinates['latt']), float(coordinates['long'])],zoom_start=15)

    for i, row in df.iterrows():
        if row["type"] == "plan":
            geom = {
            "location":[row["latitude"], row["longitude"]],
            "tooltip" : row["name"]
            }
            
            if row["subtype"] == "restaurants":
                iconito = "cutlery"
            elif row["subtype"] == "leisure":
                iconito = "rocket"
            elif row["subtype"] == "outdoors":
                iconito = "pagelines"
            elif row["subtype"] == "snacks":
                iconito = "cutlery"
            elif row["subtype"] == "party":
                iconito = "glass"
            elif row["subtype"] == "drinks":
                iconito = "glass"
            else:
                iconito = "book"

            icon = Icon(color = "lightgreen",
                            prefix = "fa",
                            icon = iconito,
                            icon_color = "black"
            )

        elif row["type"] == "transport":
            geom = {
            "location":[row["latitude"], row["longitude"]],
            "tooltip" : row["place"]
            }

            if row["place"] == "bus":            
                icon = Icon(color = "lightblue",
                                prefix = "fa",
                                icon = "bus",
                                icon_color = "black"
                )   
            elif row["place"] == "train" or row["place"] == "metro":
                icon = Icon(color = "lightblue",
                                prefix = "fa",
                                icon = "train",
                                icon_color = "black"
                )
            else:
                icon = Icon(color = "lightblue",
                                prefix = "fa",
                                icon = "car",
                                icon_color = "black"
                ) 


        Marker(**geom,icon = icon ).add_to(map_1)
    return map_1

In [44]:
df

,_id,name,latitude,longitude,geometry,place,type
0,60a24ec25065d2cc63d9ab52,AV.IGLESIA-REAL,40.491814,-3.873164,"{'type': 'Point', 'coordinates': [40.491813659...",bus,transport
1,60a24ec20ffc96451a054a87,AV.IGLESIA-REAL,40.491814,-3.873164,"{'type': 'Point', 'coordinates': [40.491813659...",bus,transport
2,60a24ec25065d2cc63d9ab81,AV.CONSTITUCIÓN-AYUNTAMIENTO,40.492527,-3.874967,"{'type': 'Point', 'coordinates': [40.492527008...",bus,transport
3,60a24ec20ffc96451a054ac6,AV.CONSTITUCIÓN-AYUNTAMIENTO,40.492527,-3.874967,"{'type': 'Point', 'coordinates': [40.492527008...",bus,transport
4,60a24ec20ffc96451a054de7,CTRA.M505-BURGOCENTRO,40.490986,-3.876587,"{'type': 'Point', 'coordinates': [40.490985870...",bus,transport
...,...,...,...,...,...,...,...
160,60a24ec20ffc96451a054e14,EMILIA PARDO BAZÁN-MARIE CURIE,40.501316,-3.881418,"{'type': 'Point', 'coordinates': [40.501316070...",bus,transport
161,60a24ec25065d2cc63d9adbe,CTRA.A6-URB.MARACAIBO,40.503811,-3.874405,"{'type': 'Point', 'coordinates': [40.503810882...",bus,transport
162,60a24ec20ffc96451a054cf3,CTRA.A6-URB.MARACAIBO,40.503811,-3.874405,"{'type': 'Point', 'coordinates': [40.503810882...",bus,transport
163,60a24ec25065d2cc63d9aba1,AV.GUADARRAMA-INSTITUTO,40.478775,-3.879287,"{'type': 'Point', 'coordinates': [40.478774557...",bus,transport


In [ ]:
mapita(df,coordinates)

In [69]:
cs